# Project: Analyzing Stock Sentiment from Twits

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. We want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, we use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. We build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. We build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

In [3]:
twits['data'][0]

{'message_body': '$FITB great buy at 26.00...ill wait',
 'sentiment': 2,
 'timestamp': '2018-07-01T00:00:09Z'}

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [4]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))


1548010


### Split Message Body and Sentiment Score

In [5]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

#reducing data size to prevent errors while training on cuda
messages=messages[:300000]
sentiments=sentiments[:300000]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. We can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [6]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'http[:./\w]+',' ',text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$[\w]+','',text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'\@[\w]+','',text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-zA-Z]',' ',text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. We can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [nltk.stem.WordNetLemmatizer().lemmatize(w) for w in tokens]

    
    assert type(tokens) == list, 'Tokens should be list'
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...


>Note: We have ensured that after preprocessing the text should NOT include:
- Numbers
- URLs
- Single character tokens
- Ticker symbols (these should be removed even if they don't appear at the beginning)

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [7]:
# TODO Implement

tokenized = [preprocess(message) for message in messages]

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [8]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

bow = Counter([x for item in tokenized for x in item])

In [9]:
len([x for item in tokenized for x in item])

3804565

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here we want to divide the count of each word by the **number of messages** calculated in the code block above (i.e. `len(messages))`. Then remove words that only appear in some small fraction of the messages.

>Note: There is not an exact number for low and high-frequency cut-offs, however there is a correct optimal range.
Ideally set up low-frequency cut-off from 0.0000002 to 0.000007 (inclusive) and high-frequency from 5 to 20 (inclusive). If the number is too big, we lose lots of important words that we can use in our data.

In [10]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = bow

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.0000002

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 20

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [w[0] for w in bow.most_common(high_cutoff)]


filtered_words = [word for word in freqs.keys() if ((freqs[word]/len(messages) > low_cutoff) and 
                                                    (word not in K_most_common))]
print(K_most_common)
len(filtered_words)

['amp', 'to', 'the', 'utm', 'a', 'for', 'is', 'on', 's', 'of', 'in', 'and', 'this', 'it', 'i', 'at', 'will', 'metric', 'up', 'source']


43272

In [11]:
[word for word in freqs.keys() if (freqs[word]/len(messages) < low_cutoff)]

[]

In [12]:
'google' in filtered_words

True

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [13]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {w:i for i,w in enumerate(filtered_words)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = dict(enumerate(filtered_words))
# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in item if word not in K_most_common] for item in tokenized]

assert set(vocab.keys()) == set(id2vocab.values()), 'Check vocab and id2vocab dictionaries'

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [14]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

In [15]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1954936472308833

Finally let's convert our tokens into integer ids which we can pass to the network.

In [16]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network we built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [17]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(self.vocab_size,self.embed_size)
        
        
        # Setup additional layers
        self.lstm = nn.LSTM(input_size=self.embed_size,
                            hidden_size=self.lstm_size,
                            num_layers=self.lstm_layers,
                            batch_first=False,
                            dropout=self.dropout)
        
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(lstm_size, output_size)
        self.activation = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),\
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
#       TODO Implement 
        
        #shape of embeds output=(seq_length,N,emdeddingsize)
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        #shape of lastm_out (L=5,N=4,h_out=6)
        lstm_out, hidden = self.lstm(embeds, hidden_state)
        #take the lstm_output corresponding to last sequence: -1 in the first dimension
        #lstm_out has dimension 4*6 where 4 is the batch size & 6 is the hidden state
        lstm_out = lstm_out[-1,:,:]
        #apply dropout to the output layer
        out = self.dropout(lstm_out)
        #apply FC Layer
        out = self.fc(out)
        #output now has size num_batches*numberof sentiments; apply final activation to yield log probabilities
        logps_out = self.activation(out)
        
        return logps_out, hidden

### View Model

In [18]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)
print(input)
logps, _ = model.forward(input, hidden)
print('\n Logits\n')
print(logps)
print('\n Probabilities\n')
print(torch.exp(logps))
#vocabsize ~ 10k
#embedsize=10
#lstim_size/hiddensize=6
#output_size=5

tensor([[ 108,  659,  617,  492],
        [ 357,  933,   51,  168],
        [ 162,  754,  861,  789],
        [ 852,  128,   14,  622],
        [ 167,   73,  171,  810]])

 Logits

tensor([[-1.2788, -1.7109, -1.4725, -1.7512, -1.9803],
        [-1.4590, -1.7689, -1.3108, -1.6312, -2.0271],
        [-1.3400, -1.7280, -1.3824, -1.7583, -1.9863],
        [-1.3351, -1.7244, -1.3825, -1.7668, -1.9894]])

 Probabilities

tensor([[ 0.2784,  0.1807,  0.2293,  0.1736,  0.1380],
        [ 0.2325,  0.1705,  0.2696,  0.1957,  0.1317],
        [ 0.2618,  0.1776,  0.2510,  0.1723,  0.1372],
        [ 0.2631,  0.1783,  0.2509,  0.1709,  0.1368]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [19]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)
    print()
    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [20]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 

train_features = token_ids[:int(0.8*len(token_ids))]
valid_features = token_ids[int(0.8*len(token_ids)):]
train_labels = sentiments[:int(0.8*len(token_ids))]
valid_labels = sentiments[int(0.8*len(token_ids)):]
#print(len(train_features))
#print(len(train_labels))
#print(len(valid_features))
#print(len(valid_labels))

In [21]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)
torch.sum(logps)

tensor(-517.3613)

### Training
It's time to train the neural network!

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(43273, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (activation): LogSoftmax()
)

In [23]:
device

device(type='cuda')

In [24]:
"""
Train  model with dropout. Clip gradients to avoid divergences in optimisation.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""
import numpy as np
epochs = 3
batch_size = 32
learning_rate = 0.001

print_every = 400
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    hidden = model.init_hidden(batch_size=batch_size)
    steps = 0
    
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        
        if text_batch.shape[1]!=batch_size:
            break
        
        steps += 1
        hidden = tuple([each.data for each in hidden])
        #print(f'step{steps}')
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        
        model.zero_grad()
        output, hidden = model(text_batch, hidden)
        loss = criterion(output, labels)
        
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        
        # End of Model Training
        
        if steps % print_every == 0:
            
            model.eval()
            val_steps=0
            
            #repeat steps for validation in eval model
            val_h = model.init_hidden(batch_size)
            val_losses=[]
            val_accuracy=[]
            
            for val_batch, val_labels in dataloader(
                    valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=False):
                
                if val_batch.shape[1]!=batch_size:
                    break
                
                val_steps+=1
                #print(f'Validation step {val_steps}')
                
                val_h = tuple([each.data for each in val_h])
                val_batch, val_labels = val_batch.to(device), val_labels.to(device)
                val_output, val_h = model(val_batch, val_h)
                #print()
                
                for each in val_h:
                    each.to(device)
                    
                val_loss = criterion(val_output, labels)
                val_losses.append(val_loss.item())
                
                #Accuracy
                val_ps = torch.exp(val_output)
                val_ps_top, val_labels_pred = val_ps.topk(1,dim=1)
                equals = val_labels_pred == val_labels.view(*val_labels_pred.shape)
                val_accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())
            
            # TODO Implement: Print metrics
            
            model.train()
            print(f'Epoch{epoch+1}/{epochs}')
            print(f'Steps {steps}')
            print(f'Training Loss: {loss.item()}')
            print(f'Validation Loss: {np.mean(val_losses)}')
            print(f'Validation accuracy: {np.mean(val_accuracy)}')
    

Starting epoch 1


Epoch1/3
Steps 400
Training Loss: 1.2408673763275146
Validation Loss: 2.0579108024031165
Validation accuracy: 0.5239075203252033

Epoch1/3
Steps 800
Training Loss: 0.7705622315406799
Validation Loss: 2.5307984525595253
Validation accuracy: 0.5763719512195122

Epoch1/3
Steps 1200
Training Loss: 0.7824156284332275
Validation Loss: 2.359941737341687
Validation accuracy: 0.6009908536585366

Epoch1/3
Steps 1600
Training Loss: 0.6723355650901794
Validation Loss: 2.298097388434216
Validation accuracy: 0.6198170731707318

Epoch1/3
Steps 2000
Training Loss: 0.807261049747467
Validation Loss: 2.531662067359056
Validation accuracy: 0.6276422764227643

Epoch1/3
Steps 2400
Training Loss: 1.017372727394104
Validation Loss: 2.6920215201571707
Validation accuracy: 0.6414126016260162

Epoch1/3
Steps 2800
Training Loss: 0.9910310506820679
Validation Loss: 2.9532981089460173
Validation accuracy: 0.640625

Epoch1/3
Steps 3200
Training Loss: 1.2150459289550781
Validation Loss: 2.58388781

## Making Predictions
### Prediction 
Okay, now that we have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [25]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in filtered_words]
    
    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.tensor(tokens).view(-1,1)
    print(text_input)
    # Get the NN output
    
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)
    
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [33]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[  340],
        [ 3206],
        [ 9651],
        [ 1164],
        [ 3031],
        [  539],
        [  155]])


tensor([[ 0.0002,  0.0023,  0.0089,  0.8781,  0.1105]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
The model suggests that the prediction for test "Google is working on self driving cars, I'm bullish on $goog", belongs to sentiment 3 (in scaled terms) with a prediction probability of ~88%

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, I'd want to track the sentiments of the stocks in my universe and use this as a signal in larger pipelines.

## Testing
### Load the Data 

In [34]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [35]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [36]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [37]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

tensor([[  115],
        [  116],
        [  117],
        [   34],
        [  118],
        [  119],
        [ 1711]])
tensor([[ 16845],
        [   263],
        [   200],
        [  5687],
        [  2008],
        [   127],
        [   134],
        [   910],
        [ 16845],
        [    57],
        [  3044],
        [   549],
        [   110],
        [    45],
        [   316],
        [  1036],
        [   124]])
tensor([[   61],
        [ 1413]])


{'symbol': '$AAPL',
 'score': tensor([[ 0.2691,  0.3062,  0.0678,  0.2385,  0.1184]]),
 'timestamp': '2018-11-01T00:00:18Z'}